In [29]:
!pip install nltk
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords #Word Stop
from nltk.tokenize import word_tokenize #Tokenization & Word Stop
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation + '``'+ '`'+ ''+ ',' + '/')
import joblib
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rpierson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
#Define a Preprocessing subpipeline
class PreprocessingPineline:
    def __init__(self, stop_words, punctuation):
        self.punctuation = punctuation
        self.stop_words = stop_words
        self.data = None
        self.tokens = None
    def data_to_tokens(self, data):
        self.data = data
        self.tokens = self.data.astype(str).apply(word_tokenize)
        self.tokens = [[word for word in tokens if word.lower() not in stop_words and word not in punctuation] for tokens in self.tokens]
        return self.tokens
    
preprocess = PreprocessingPineline(stop_words, punctuation)

In [31]:
#Define a Topic Modeling subPipeline
class LDATopicModelPipeline:
    def __init__(self, lda_model_path, vectorizer_path):
        self.lda_model_path = lda_model_path
        self.vectorizer_path = vectorizer_path
        self.lda = None
        self.vectorizer = None
        
    def load_model(self):
        self.lda = joblib.load(self.lda_model_path)
        self.vectorizer = joblib.load(self.vectorizer_path)
        
    def topic_distributions(self, new_documents):
        if self.lda is None or self.vectorizer is None:
            self.load_model()
            
        docs_new = self.vectorizer.transform(new_documents)
        topic_distributions = self.lda.transform(docs_new)
        return topic_distributions

lda_model_path = '/home/rpierson/PiersonREU/extracted/lda.pkl'
vectorizer_path = '/home/rpierson/PiersonREU/extracted/vec.pkl'
#gives necessary files to process information
topic_model = LDATopicModelPipeline(lda_model_path, vectorizer_path)
topic_model.load_model()
#use through this syntax: topic_distributions = topic_model.topic_distributions(new_documents)

In [32]:
#Define an Text Classification subpipeline
class TextClassificationNaiveBayes:
    def __init__(self, bayes_path):
        self.documents = None
        self.num_topics = None
        self.bayes_path = bayes_path
    
    def load_model(self):
        self.bayes_trained = joblib.load(self.bayes_path)
        
    def priority(self):

SyntaxError: incomplete input (2149994588.py, line 11)

In [34]:
#Using the equations for accuracy information
class AccuracyAssessment:
    def __init__(self, priority_levels):
        self.actual_priority = None
        self.predicted_priority = None
        self.true_pos = None
        self.false_pos = None
        self.false_neg = None
        self.num_priority_levels = priority_levels
        self.confusion_matrix = np.zeros((self.num_priority_levels, self.num_priority_levels), dtype=int)
    
    def update_vals(self, actual, predicted):
        self.actual_priority = actual
        self.predicted_priority = predicted
        for i in range(len(self.actual_priority)):
            self.confusion_matrix[self.actual_priority[i] - 1, self.predicted_priority[i] - 1] += 1
    
    def calc_metrics(self, class_index):
        self.true_pos = (actual_priority == predicted_priority).sum()
        self.false_pos = np.sum(self.confusion_matrix[:, class_index]) - self.true_pos
        self.false_neg = np.sum(self.confusion_matrix[class_index, :]) - self.true_pos
    
    def precision(self, class_index):
        AccuracyAssessment.calc_metrics(class_index)
        
        if self.true_pos + self.false_pos == 0:
            return 0
        precision = self.true_pos / (self.true_pos + self.false_pos)
        
        return precision
    
    def recall(self, class_index):
        AccuracyAssessment.calc_metrics(class_index)
        
        if self.true_pos + self.false_neg == 0:
            return 0
        
        recall = self.true_pos / (self.true_pos + self.false_neg)
        
        return recall
    
    def fmeasure(self, class_index):
        precision = AccuracyAssessment.precision(class_index)
        recall = AccuracyAssessment.recall(class_index)
        
        if precision + recall == 0:
            return 0
        
        fmeasure = (2 * precision * recall) / (precision + recall)
        
        return fmeasure
    
    def printAssessment(self):
        for i in range(self.num_priority_levels):
            precision_score = AccuracyAssessment.precision(i)
            recall_score = AccuracyAssessment.recall(i)
            f_measure_score = AccuracyAssessment.fmeasure(i)
    
            print(f"Class {i+1}: Precision={precision_score:.4f}, Recall={recall_score:.4f}, F-measure={f_measure_score:.4f}")

assessment = AccuracyAssessment(priority_levels = 5)
#make sure the rows are formatted appropriately for the following:
#assessment.update_vals(actual, predicted)

In [ ]:
def custom_pipeline(data):
    #PreProcessing
    #Topic Modeling (Insert Trained Model Here, Save as a .pth)
    #Text Classification Per Topic (Insert Trained Model Here, Save as a .pth)
    #Accuracy and Evaluation